# Requirems

### 1. Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install gradio
!pip install --upgrade gradio
# When installation is installed, after restarting the cell, skip this cell and start
!pip install PyPortfolioOpt
!pip install yfinance
!pip install matplotlib
import pandas as pd
import numpy as np
import yfinance as yf
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
import gradio as gr
from pypfopt import EfficientFrontier, risk_models, BlackLittermanModel, expected_returns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2. Data pretreatment

In [ ]:
#Destructive function definition
def preprocess_data(df):
    if 'environmental' in df.columns and 'social' in df.columns and 'governance' in df.columns:
        df['env_percent'] = df['environmental'] / (df['environmental'] + df['social'] + df['governance'])
        df['soc_percent'] = df['social'] / (df['environmental'] + df['social'] + df['governance'])
        df['gov_percent'] = df['governance'] / (df['environmental'] + df['social'] + df['governance'])
        df['env_score'] = df['average_label'] * df['env_percent']
        df['soc_score'] = df['average_label'] * df['soc_percent']
        df['gov_score'] = df['average_label'] * df['gov_percent']

        latest_year = df['Year'].max()
        year_weights = {
            latest_year: 0.5,
            latest_year - 1: 0.25,
            latest_year - 2: 0.125,
            latest_year - 3: 0.0625,
            latest_year - 4: 0.0625
        }

        df['environmental'] = df.apply(lambda x: x['env_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['social'] = df.apply(lambda x: x['soc_score'] * year_weights.get(x['Year'], 0), axis=1)
        df['governance'] = df.apply(lambda x: x['gov_score'] * year_weights.get(x['Year'], 0), axis=1)

        final_df = df.groupby(['Company', 'industry', 'ticker']).agg({
            'environmental': 'sum',
            'social': 'sum',
            'governance': 'sum'
        }).reset_index()

        return final_df
    else:
        raise KeyError("The expected columns 'environmental', 'social', and 'governance' are not present in the dataframe.")

# Data pretreatment execution
dummy = pd.read_csv('/content/drive/MyDrive/Kwargs/241007_dummy_update_static.csv', encoding='cp949')
dummy = preprocess_data(dummy)

### 3. Optimization function

In [ ]:
# Portfolio weight calculation function with black-litterman
from pypfopt import CovarianceShrinkage

# Modified portfolio weight calculation function with black-litterman and reduced covariance matrix
def calculate_portfolio_weights_with_shrinkage(df, esg_weights, user_investment_style):
    tickers = df['ticker'].tolist()
    price_data = yf.download(tickers, start="2019-01-01", end="2023-01-01")['Adj Close']
    price_data = price_data.dropna(axis=1)  # 결측치 제거

    if price_data.isnull().values.any():
        return "일부 데이터가 누락되었습니다. 다른 기업을 선택해 주세요.", None

# Average return and covariance matrix
    mu_market = expected_returns.capm_return(price_data)

# Application of reduction of covenant matrix
    Sigma = CovarianceShrinkage(price_data).ledoit_wolf()

# ESG score adjustment reflecting user preference
    df['final_esg_score'] = (
        esg_weights['environmental'] * df['environmental'] +
        esg_weights['social'] * df['social'] +
        esg_weights['governance'] * df['governance']
    )

# ESG score weight according to user disposition
    if user_investment_style == "재무지표 중심":
        esg_weight_factor = 0.5  # 안정형: ESG 점수 가중치를 낮게 반영
    elif user_investment_style == "ESG-재무 절충":
        esg_weight_factor = 1  # 위험 중립형: 중간 반영
    elif user_investment_style == "ESG 점수 중심":
        esg_weight_factor = 1.5  # 공격형: ESG 점수 가중치를 높게 반영

# Reflecting the tendency in the final ESG score
    df['adjusted_esg_score'] = df['final_esg_score'] * esg_weight_factor
    valid_tickers = price_data.columns.tolist()
    df_valid = df[df['ticker'].isin(valid_tickers)]

# Investor's opinion reflects
    P = np.eye(len(valid_tickers))
    Q = df_valid['adjusted_esg_score'].values

# Black-Litterman model application
    bl = BlackLittermanModel(Sigma, pi=mu_market, P=P, Q=Q)
    adjusted_returns = bl.bl_returns()

# Optimization Problem Setting
    n = len(mu_market)
    P_opt = matrix(Sigma.values)
    q_opt = matrix(-adjusted_returns.values)
# Add optimization constraints to limit the proportion of portfolios
    G = matrix(np.vstack([-np.eye(n), np.eye(n)]))  # 부등식 제약조건: 각 자산 가중치가 0 이상
    h = matrix(np.hstack([np.zeros(n), np.ones(n) * 0.1]))  # 각 자산의 비중을 최대 10%로 제한

    G = matrix(-np.eye(n))
    h = matrix(np.zeros(n))
    A = matrix(1.0, (1, n))
    b = matrix(1.0)

# Optimal weight calculation
    sol = solvers.qp(P_opt, q_opt, G, h, A, b)
    weights = np.array(sol['x']).flatten()

# Calculation of portfolio performance
    expected_return = np.dot(weights, mu_market)
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(Sigma.values, weights)))
    sharpe_ratio = expected_return / expected_volatility

#Brubilion
    cleaned_weights = dict(zip(valid_tickers, weights))

    return cleaned_weights, (expected_return, expected_volatility, sharpe_ratio)

# This function is ready to use as a Gradio interface.

### 4.gradio -be

In [ ]:

# Gradio interface function
def gradio_interface(environmental, social, governance, investment_style):
    esg_weights = {'environmental': environmental, 'social': social, 'governance': governance}

# Treatment of all companies
    processed_df = dummy.copy()

# Portfolio weight calculation
    portfolio_weights, portfolio_performance = calculate_portfolio_weights_with_shrinkage(processed_df, esg_weights, investment_style)

    if portfolio_weights is None:
        return "No portfolio weights could be calculated. Please adjust your input values.", None

#Sorting portfolio weight data in the order of large proportion
    sorted_weights = dict(sorted(portfolio_weights.items(), key=lambda item: item[1], reverse=True))

# Drawing a pie chart for the portfolio weighting visualization
    fig, ax = plt.subplots()
    colors = plt.get_cmap("Set3").colors  # Set3 컬러맵 사용
    ax.pie(
        list(sorted_weights.values()),
        labels=list(sorted_weights.keys()),
        colors=colors[:len(sorted_weights)],
        autopct='%1.1f%%',
        startangle=90,  # 12시 방향 시작
        counterclock=True  # 반시계 방향으로 그리기
    )
    ax.set_title('Calculated Portfolio')
    ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1))

#Fortfolio performance text results return
    performance_text = (
        f"🟢 **Expected Annual Return**: {portfolio_performance[0]:.2%}\n"
        f"🔵 **Annual Volatility**: {portfolio_performance[1]:.2%}\n"
        f"🟠 **Sharpe Ratio**: {portfolio_performance[2]:.2f}"
    )
    return performance_text, fig

### 5. Gradio -FE

In [ ]:
#Layout setting using Gradio Blocks API
with gr.Blocks() as gr_interface:
# Input with sliders and radio buttons
    with gr.Row():
       with gr.Column():
           environmental_slider = gr.Slider(0, 0.01, value=0, label='Environmental')
           social_slider = gr.Slider(0, 0.01, value=0, label='Social')
           governance_slider = gr.Slider(0, 0.01, value=0, label='Governance')
           investment_style_radio = gr.Radio(choices=["재무지표 중심", "ESG-재무 절충", "ESG 점수 중심"], label="투자 성향 선택")
           submit_button = gr.Button("Submit")


# Output layout setting
       with gr.Column():
           performance_output = gr.Textbox(label="Portfolio Performance")
           portfolio_plot = gr.Plot(label="Portfolio Allocation")

# Call function when clicking the button
    submit_button.click(
        gradio_interface,
        inputs=[environmental_slider, social_slider, governance_slider, investment_style_radio],
        outputs=[performance_output, portfolio_plot]
    )

# Gradio Activate

In [ ]:
# Run Gradio Interface
gr_interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1f908d8aab4af07722.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[*********************100%***********************]  67 of 67 completed


     pcost       dcost       gap    pres   dres
 0: -4.3204e-02 -1.0523e+00  1e+00  0e+00  8e+00
 1: -4.3229e-02 -6.2229e-02  2e-02  3e-17  1e-01
 2: -4.3951e-02 -4.8995e-02  5e-03  1e-16  3e-02
 3: -4.4495e-02 -4.5199e-02  7e-04  2e-16  8e-05
 4: -4.4568e-02 -4.4652e-02  8e-05  3e-17  6e-06
 5: -4.4583e-02 -4.4591e-02  8e-06  3e-17  2e-07
 6: -4.4585e-02 -4.4586e-02  7e-07  4e-17  8e-09
 7: -4.4585e-02 -4.4585e-02  4e-08  1e-16  1e-10
Optimal solution found.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1f908d8aab4af07722.gradio.live
